# Исследование надёжности заёмщиков


Входные данные — данные о клиентах банка и их займах. Задача - выяснить влияет ли семейное положение и количество детей, а также другие факторы на частоту погашения кредита в срок, результаты исследования будут учитываться кредитным отделом для построения модели кредитного скоринга.

## Общая информация

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

В таблице 12 столбцов, тип данных object, int, float.
Согласно документации к данным:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Количество значений в некоторых столбцах различается, значит есть пропуски. Стиль заголовков соблюдается. В столбце days_employed некоторые значения отрицательные. В столбце education есть неявные дубликаты.  


## Предобработка данных

В столбце days_employed некоторые значения отрицательные, такая ситуация может быть результатом технической ошибки или ошибкой ввода. Исправим значения на положительные.

In [4]:
data['days_employed'] = data['days_employed'].abs()

В столбце days_employed некоторые значения очень большие. Найдем максимальное значение столбца.

In [5]:
print(data['days_employed'].max())

401755.40047533


Т.е. 401755/365 = 1100 лет, вероятно это значение приведено не в днях, а в часах. 401755/24 = 16425 дней/365 = 45 лет. Если считать начало трудового стажа с 20 лет, потенциальный максимально возможный стаж около 60 лет = 21900 дней.Таким образом для преобразования значений в дни, все переменные, которые больше 21900 поделим на 24.

In [6]:
# функция для перевода часов в дни
def to_days(value):
    if value<21900:
        return value
    if value>21900:
        return value/24
data['days_employed'] = data['days_employed'].apply(to_days)
# Проверим
print(data['days_employed'].max())

18388.949900568383


Теперь все данные о стаже представлены в днях. Можно перейти к заполнению пропусков.

### Обработка пропусков

В столбцах days_employed и total_income данные пропущены. 

**days_employed**

In [7]:
print('Пропуски до:',data['days_employed'].isna().sum())

Пропуски до: 2174


Возможно некоторые заёмщики не знали свой трудовой стаж в днях, поэтому не предоставляли информацию. Исключение этих строк может привести к существенному искажению результата. Данные в этих столбцах являются количественными переменными, поэтому мы можем заменить пропуски характерными значениями для этой выборки. Для замены подойдет медиана выборки.

Для более точного отражения медианы выборки целесообразно расчитать медианы стажа с группировкой по другим переменным. Для стажа наиболее подходящей будет переменная gender и education. Так например, в группе мужчин медиана стажа будет выше чем в группе женщин, люди с более низким уровнем образования чаще начинают работать в более раннем возрасте, а высококвалифицированные специалисты могут оставаться востребованы и по достижении пенсионного возраста.

В столбце education неявные дубликаты за счет разного регистра. Приведем значения в столбце к нижнему регистру.

In [8]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [9]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Заполнять характерными значениями можно только пропуски случайного характера. Учитывая проведение подсчета медианы в зависимости от других переменных, проверим является ли пропуски случайными по отношению к этим столбцам.

In [10]:
# выделим строки с пропусками в days_employed
days_empl_pass = data[data['days_employed'].isna()== True]
#  распределение значений для столбца gender
print(days_empl_pass['gender'].value_counts(normalize = True))
print()
# выделим строки без пропусков days_employed
days_empl_full = data[data['days_employed'].isna()== False]
# распределение значений для столбца income_type
print(days_empl_full['gender'].value_counts(normalize = True))
print()
# для столбца education
print(days_empl_pass['education'].value_counts(normalize = True))
# оценим распределение значений для столбца income_type
days_empl_full['education'].value_counts(normalize = True)

F    0.682613
M    0.317387
Name: gender, dtype: float64

F      0.658984
M      0.340964
XNA    0.000052
Name: gender, dtype: float64

среднее                0.708372
высшее                 0.250230
неоконченное высшее    0.031739
начальное              0.009660
Name: education, dtype: float64


среднее                0.707612
высшее                 0.243708
неоконченное высшее    0.034882
начальное              0.013488
ученая степень         0.000310
Name: education, dtype: float64

С точностью до десятых распределение в столбцах совпадает, таким образом пропуски от этого столбца не зависят.

In [11]:
# создадим датафрейм с группировкой по gender и education и вычислим медиану в столбцах days_employed
medians = (data.groupby(['gender', 'education']).agg({'days_employed':'median'}).rename(columns = {'days_employed':'median_days_employed'}))
# объединяем два датафрейма получаем новый столбец median_days_employed
data = data.merge(medians, on = ['gender', 'education'])
data[['gender', 'education','days_employed','median_days_employed']].head()

,gender,education,days_employed,median_days_employed
0,F,высшее,8437.673028,2009.211057
1,F,высшее,2879.202052,2009.211057
2,F,высшее,6929.865299,2009.211057
3,F,высшее,1844.956182,2009.211057
4,F,высшее,717.274324,2009.211057


In [12]:
# заменим значения days_employed, где они отсутсвуют на median_days_employed
data.loc[data['days_employed'].isna(), 'days_employed'] = data.loc[data['days_employed'].isna(), 'median_days_employed']

# проверим
print('Пропуски после:', data['days_employed'].isna().sum())

Пропуски после: 0


**total_income**

Рассчитаем медиану выборки total_income в зависимости от типа занятости. Поскольку в группах с зарплатой, пособием, пенсией и стипендией медианы будут отличаться.

Прежде чем заполнять пропуски нужно убедиться, что они носят случайный характер и не зависят от значений типа занятости.

In [13]:
# выделим строки с пропусками в total_income
income_pass = data[data['total_income'].isna()== True]
# оценим распределение значений для столбца income_type
print(income_pass['income_type'].value_counts(normalize = True))
# выделим строки без пропусков total_income
income_full = data[data['total_income'].isna()== False]
# оценим распределение значений для столбца income_type
income_full['income_type'].value_counts(normalize = True)

сотрудник          0.508280
компаньон          0.233671
пенсионер          0.189972
госслужащий        0.067617
предприниматель    0.000460
Name: income_type, dtype: float64


сотрудник          0.517493
компаньон          0.236525
пенсионер          0.177924
госслужащий        0.067800
безработный        0.000103
предприниматель    0.000052
студент            0.000052
в декрете          0.000052
Name: income_type, dtype: float64

С точностью до десятых распределение в столбце в срезах совпадает, таким образом пропуски от этого столбца не зависят.

In [14]:
print('Пропуски до:',data['total_income'].isna().sum())

# заменим значения total_income, где они отсутсвуют на медиану days_employed в соответствии с типом занятости
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x : x .fillna(x.median()))

# проверим
print('Пропуски после:', data['total_income'].isna().sum())

Пропуски до: 2174
Пропуски после: 0


**Вывод**

Потеря большого количества данных из-за пропусков может существенно отразиться на репрезентативности выборки, заменяя количественные переменные на характеристические с учетом влияния зависимости от других переменных, можно сохранить поведение выборки. 

### Замена типа данных

В столбце days_employed тип данных вещественные числа, стаж измеряется в целых днях, необходимо изменить тип данных.

In [15]:
data['days_employed'] = data['days_employed'].astype('int')
# в столбце total_income данные могут быть float, но для удобства восприятия также переведем в int
data['total_income'] = data['total_income'].astype('int')
data[['days_employed','total_income']].head()

,days_employed,total_income
0,8437,253875
1,2879,240525
2,6929,95856
3,1844,165127
4,717,187863


**Вывод**

Нет необходимости в точности измерений до шестого знака после запятой, а таблица для восприятия и последующей обработки стала удобнее.

### Обработка дубликатов

Проверим данные на наличие явных дубликатов.

In [16]:
print('Дубликаты до:', data.duplicated().sum())

Дубликаты до: 71


In [17]:
# Удалим явные дубликаты
data = data.drop_duplicates().reset_index(drop = True)
# Проверим
print('Дубликаты после:', data.duplicated().sum())

Дубликаты после: 0


Проверим столбцы на наличие скрытых дубликатов

In [18]:
print(data['children'].unique())
# Потенциально у человека может быть 20 детей, посмотрим сколько таких счастливчиков
data[data ['children'] == 20]['children'].count()

[ 1  0  2  4  3 20 -1  5]


76

Слишком много, больше похоже на ошибку, 0 напечатан случайно, клавиши близко. Исправим. К тому же, есть значения -1 .Вероятно произошла случайная ошибка и имеется в виду отсутсвие детей. Исправим.

In [19]:
#def child_count(value):
    #if value == 20:
        #return 2
    #elif value == -1:
        #return 0
    #else:
        #return value
#data['children'] = data['children'].apply(child_count)
data['children'] = data['children'].replace({20 : 2, -1 : 0})
# Проверим
print(data['children'].unique())

[1 0 2 4 3 5]


In [20]:
data['dob_years'].unique()

array([42, 43, 35, 56, 26, 31, 50, 27, 52, 63, 40, 34, 51, 47, 54, 24, 33,
       38, 29, 55, 64, 37, 41, 39, 46, 60, 59, 22, 44, 68, 36, 28, 30, 45,
       49, 65, 57, 53, 62, 25, 23, 58, 32, 48, 61,  0, 69, 67, 21, 20, 66,
       70, 72, 74, 71, 73, 19, 75])

В столбце возраста есть значение 0, так как это количественная переменная можно заменить на медиану. 

Но прежде чем заменять значение медианой, проверим не относятся ли эти пропуски к группам с определенными возрастными интервалами, например, пенсионеры или студенты.

In [21]:
# выделим строки с 0 в dob_years
years_null = data[data['dob_years'] == 0]
# оценим распределение значений для столбца income_type
print(years_null['income_type'].value_counts(normalize = True))

сотрудник      0.544554
компаньон      0.198020
пенсионер      0.198020
госслужащий    0.059406
Name: income_type, dtype: float64


Пропуски относятся к разным группам, медиана подойдет для замены.

In [22]:
median_years = data['dob_years'].median().astype('int')
data.loc[data['dob_years'] == 0, 'dob_years'] = median_years
# Проверим
data['dob_years'].unique()

array([42, 43, 35, 56, 26, 31, 50, 27, 52, 63, 40, 34, 51, 47, 54, 24, 33,
       38, 29, 55, 64, 37, 41, 39, 46, 60, 59, 22, 44, 68, 36, 28, 30, 45,
       49, 65, 57, 53, 62, 25, 23, 58, 32, 48, 61, 69, 67, 21, 20, 66, 70,
       72, 74, 71, 73, 19, 75])

**Вывод**

Поиск и устранение явных и неявных дубликатов структурирует данные, позволяя избежать влияния ошибок ввода данных на последующий анализ.

### Лемматизация

In [23]:
data['purpose'].unique()

array(['покупка жилья', 'операции с жильем', 'на проведение свадьбы',
       'покупка жилой недвижимости',
       'строительство собственной недвижимости',
       'приобретение автомобиля', 'заняться образованием', 'свадьба',
       'автомобили', 'покупка жилья для семьи', 'свой автомобиль',
       'жилье', 'сыграть свадьбу', 'автомобиль', 'профильное образование',
       'покупка жилья для сдачи', 'сделка с подержанным автомобилем',
       'операции со своей недвижимостью',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'на покупку своего автомобиля',
       'высшее образование', 'на покупку подержанного автомобиля',
       'покупка своего жилья', 'покупка недвижимости',
       'операции с недвижимостью', 'образование',
       'заняться высшим образованием', 'получение высшего образования',
       'ремонт жилью', 'недвижимость', 'строительство недвижимости',
       'покупка коммерческой недвижимости', 'дополнительное образование',
       'по

В столбце purpose много неявных дубликатов из-за форм записи слов, синонимов, орфографических ошибок. Используем лемматизацию, чтобы систематизировать подобные данные.

In [24]:
# найдем все уникальные значения столбца
purpose = data['purpose'].unique()

In [25]:
purpose_lemma = []
for value in purpose:
    lemma =  m.lemmatize(value)
    purpose_lemma.extend(lemma)
     
print(purpose_lemma)

['покупка', ' ', 'жилье', '\n', 'операция', ' ', 'с', ' ', 'жилье', '\n', 'на', ' ', 'проведение', ' ', 'свадьба', '\n', 'покупка', ' ', 'жилой', ' ', 'недвижимость', '\n', 'строительство', ' ', 'собственный', ' ', 'недвижимость', '\n', 'приобретение', ' ', 'автомобиль', '\n', 'заниматься', ' ', 'образование', '\n', 'свадьба', '\n', 'автомобиль', '\n', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n', 'свой', ' ', 'автомобиль', '\n', 'жилье', '\n', 'сыграть', ' ', 'свадьба', '\n', 'автомобиль', '\n', 'профильный', ' ', 'образование', '\n', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'сдача', '\n', 'сделка', ' ', 'с', ' ', 'подержанный', ' ', 'автомобиль', '\n', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n', 'строительство', ' ', 'жилой', ' ', 'недвижимость', '\n', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n', 'высокий', ' ', 'образование', '\n', 'на', ' ', 'покупка', ' ', 'подержать

**Вывод**

Мы получили список всех уникальных лемм столбца purpose. Посчитаем частоту их встречаемости и выделим основные для проведения категоризации.

### Категоризация данных

In [26]:
from collections import Counter
Counter(purpose_lemma) 

Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         '\n': 38,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'свадьба': 3,
         'жилой': 2,
         'недвижимость': 10,
         'строительство': 3,
         'собственный': 1,
         'приобретение': 1,
         'автомобиль': 9,
         'заниматься': 2,
         'образование': 9,
         'для': 2,
         'семья': 1,
         'свой': 4,
         'сыграть': 1,
         'профильный': 1,
         'сдача': 1,
         'сделка': 2,
         'подержанный': 1,
         'со': 1,
         'коммерческий': 2,
         'высокий': 3,
         'подержать': 1,
         'получение': 3,
         'ремонт': 1,
         'дополнительный': 2})

Леммами для категоризации будут: недвижимость/жилье, автомобиль, образование, свадьба, 

In [27]:
# напишем функцию для категоризации с учетом лемм        
from functools import lru_cache
@lru_cache(maxsize=128, typed=False)
def categorize(purpose):
    wrong_lines_content = []
    lemmas = m.lemmatize(purpose)
    if 'недвижимость' in lemmas or 'жилье' in lemmas:
        return 'недвижимость'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'

data['purpose_category'] = data['purpose'].apply(categorize)
print(data[['purpose', 'purpose_category']].head(10))
# проверим всем ли значениям присвоена категория, нет ли пропусков
data['purpose_category'].unique()

                                  purpose purpose_category
0                           покупка жилья     недвижимость
1                       операции с жильем     недвижимость
2                   на проведение свадьбы          свадьба
3              покупка жилой недвижимости     недвижимость
4  строительство собственной недвижимости     недвижимость
5                 приобретение автомобиля       автомобиль
6                   заняться образованием      образование
7                                 свадьба          свадьба
8                              автомобили       автомобиль
9                 покупка жилья для семьи     недвижимость


array(['недвижимость', 'свадьба', 'автомобиль', 'образование'],
      dtype=object)

**Вывод**

После категоризации данные стали более удобными для анализа показателей в зависимости от целей кредита.

## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [33]:
# общая доля должников в выборке
total_ratio = data['debt'].sum()/data['debt'].count()
print('Общая доля должников в выборке','{:.2}%'.format(total_ratio))
# составим сводную таблицу с группировкой по столбцу children
report_1 = data.pivot_table(index = ['children'], values = 'debt', aggfunc = ['sum', 'count','mean'])
report_1.columns = ['debt', 'total', '%']
report_1 = report_1.sort_values(by = ['%'], ascending = False)
report_1.style.format({'%': '{:.2%}'})

Общая доля должников в выборке 0.081%


,debt,total,%
children,,,
4,4,41,9.76%
2,202,2128,9.49%
1,444,4808,9.23%
3,27,330,8.18%
0,1064,14138,7.53%
5,0,9,0.00%


**Вывод**

В целом процент задолженностей в группах людей  с разным количеством детей незначительно различается. Самым "лучшим" и "худшим" значением процента задолженностей являются люди с 5 и 4 детьми соответсвенно, но эти выборки очень малы, поэтому не могут быть репрезентативными. Оценивая остальные значения отмечается, что процент возврата кредита в срок людьми без детей или с 3 детьми выше, чем у людей с 1 или 2 детьми. К тому же процент задолженности по кредиту у людей с 1 и 2 детьми выше, чем в среднем по выборке.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [29]:
print('Общая доля должников в выборке','{:.2}%'.format(total_ratio))
# составим сводную таблицу с группировкой по столбцу family_status
report_2 = data.pivot_table(index = ['family_status'], values = 'debt', aggfunc = ['sum', 'count','mean'])
report_2.columns = ['debt', 'total', '%']
report_2= report_2.sort_values(by = ['%'], ascending = False)
report_2.style.format({'%': '{:.2%}'})  


Общая доля должников в выборке 0.081%


,debt,total,%
family_status,,,
Не женат / не замужем,274,2810,0.098%
гражданский брак,388,4151,0.093%
женат / замужем,931,12339,0.075%
в разводе,85,1195,0.071%
вдовец / вдова,63,959,0.066%


**Вывод**

Прослеживается зависимость между семейным положением и возвратом кредита в срок. Согласно данным анализа вероятность возврата кредита в срок выше у людей в разводе и вдовцов/вдов, к тому же показатели в этих группах меньше, чем в среднем в выборке, а в группе людей, состоящих в браке и проживающих в гражданском браке, процент задолженностей по кредиту значительно выше, также выше среднего процента по выборке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Прежде чем оценивать влияние доходов, разделим значения доходов на интервалы и присвоим им категории

Разделим на интервалы:

In [30]:
def categorize_income(income):
    if income < 100000:
        return '< 100.000'
    if (income>100000) and (income < 150000):
        return '100000-150000'
    if (income>150000) and (income < 200000):
        return '150000-200000'
    if (income>200000) and (income < 600000):
        return '200000-600000'
    if income > 600000:
        return '>600000'
data['income_category'] = data.loc[:,'total_income'].apply(categorize_income)
data[['total_income', 'income_category']].tail()

,total_income,income_category
21449,268411,200000-600000
21450,115949,100000-150000
21451,203905,200000-600000
21452,111392,100000-150000
21453,255425,200000-600000


In [31]:
print('Общая доля должников в выборке','{:.2}%'.format(total_ratio))
# составим сводную таблицу с группировкой по столбцу income_category
report_3 = data.pivot_table(index = ['income_category'], values = 'debt', aggfunc = ['sum', 'count','mean'])
report_3.columns = ['debt', 'total', '%']
report_3 = report_3.sort_values(by = ['%'], ascending = False)
report_3.style.format({'%': '{:.2%}'})

Общая доля должников в выборке 0.081%


,debt,total,%
income_category,,,
100000-150000,624,7160,0.087%
150000-200000,405,4764,0.085%
< 100.000,354,4463,0.079%
>600000,8,109,0.073%
200000-600000,350,4958,0.071%


**Вывод**

В целом процент задолженностей в группах людей с разным уровнем доходов незначительно различается. Выборка с доходом более 600 тыс. малочисленна, поэтому не может быть репрезентативна. Оценивая остальные значения отмечается, что процент возврата кредита в срок людьми с доходом 200-600 тыс. и менее 100 тыс. выше, притом частота задолженностей меньше, чем средняя по выборке. У людей с доходами до 100 и до 600 тысяч. вероятность задолженности выше, но сопоставима со средней по выборке.

- Как разные цели кредита влияют на его возврат в срок?

In [32]:
print('Общая доля должников в выборке','{:.2}%'.format(total_ratio))
# составим сводную таблицу с группировкой по столбцу purpose_category
report_4 = data.pivot_table(index = ['purpose_category'], values = 'debt', aggfunc = ['sum', 'count','mean'])
report_4.columns = ['debt', 'total', '%']
report_4 = report_4.sort_values(by = ['%'], ascending = False)
report_4.style.format({'%': '{:.2%}'})

Общая доля должников в выборке 0.081%


,debt,total,%
purpose_category,,,
автомобиль,403,4306,0.094%
образование,370,4013,0.092%
свадьба,186,2324,0.08%
недвижимость,782,10811,0.072%


**Вывод**

Наблюдается зависимость между целью кредита и возвратом его в срок. Так вероятность задолженности выше у клиентов с целью кредита, связанной с автомобилями и образованием, их значения практически одинаковы и превышают среднюю по выборке вероятность задолженности. Кредиты на недвижимость возвращают в срок с частотой сопоставимой со средней частотой выборки. Процент задолженностей по кредитам на свадьбу - минимальный.

##  Общий вывод

Наиболее достоверными оказалось влияние на возврат кредита в срок целей кредита, количества детей и семейного положения занимателя. Уровень дохода не дал значимых различий по влиянию на возврат в срок, при этом все значения сопоставимы со средней по выборке. Предикторами более высокого риска задолженности по кредиту являются:
* 1 и 2 детей
* наличие официального или гражданского супруга
* получение кредита на автомобиль или образование